In [1]:
import os
WORK = os.environ["WORK"]
PROJ_DIR = f'{WORK}/ADA_Project'
os.chdir(PROJ_DIR)

from glob import glob
from tqdm import tqdm
from pathlib import Path
import shutil
from pprint import pprint
from datetime import datetime
import subprocess
import random
import PIL
import matplotlib.pyplot as plt
import base64
import requests
import dotenv
from datetime import datetime
from IPython.display import Markdown, display

In [2]:
def generate_latest_fakes_report(PROJ_DIR, verbose=1):
    
    def upload_img(image, token):
        with open(image, "rb") as file:
            url = "https://api.imgbb.com/1/upload"
            parameters = {
                "key": token,
                "image": base64.b64encode(file.read()),
            }
            res = requests.post(url, parameters)
            link = res.json()
            url = link['data']['url']
            return url
        
    dotenv.load_dotenv(f'{PROJ_DIR}/.env')
    token = os.getenv('TOKEN')

    mb_size = lambda x: Path(x).stat().st_size / (1024*1024)
    dir_up = lambda x, y: "/".join(Path(x).parts[y:])
    
    WORK = Path(PROJ_DIR).parent
    TRfolders_ = f'{PROJ_DIR}/training_runs'
    TRfolders = glob(f'{TRfolders_}/*')

    md_content = []
    latest_fakes = []
    
    now = datetime.now()
    date_time = now.strftime('%m/%d/%Y, %H:%M:%S')
    md_content.append('# Latest fakes\n')
    md_content.append(f'## Date and time: {date_time}\n')

    for folder in TRfolders:
        for num in range(-1, -10, -1):
            files = sorted(glob(folder + "/**/*"))
            files = [x for x in files if 'fakes0' in x]
            if files == []:
                continue
            else:
                latest_fake = sorted(files)[-1]
                latest_fakes.append(latest_fake)
                break
    
    if verbose == 1:
        print('=' * 90,
             '\n\nLatest fakes:\n')
        pprint([x.replace(str(TRfolders_), '') for x in latest_fakes])
        print('\n', '=' * 90, '\n')

    for img in latest_fakes:
        image = PIL.Image.open(img)
        resized_path = f'{PROJ_DIR}/.tmp_imgs/{Path(img).name}'
        output_dim = tuple(x // 4 for x in image.size)
        resized = image.resize(output_dim)
        resized.save(resized_path)
        if verbose == 1:
            print(Path(img).name,
                f'Resized from {image.size} [{mb_size(img):.2f}MB] to ==> '
                  f'{output_dim} [{mb_size(resized_path):.2f}MB]')
        
        url = upload_img(resized_path, token)
        if verbose == 1:
            print(f'Link ==> {url}\n')
        img_subdir = dir_up(img, -3)
        
        md_content.append(f'### {img_subdir}\n'
              f'![{Path(resized_path).name}]({url} "{img_subdir}")'
              '\n\n')
    
    Tstamp = datetime.now().strftime('%m_%d_%Y__%H_%M')
    report_path = f'{PROJ_DIR}/latest_fakes_markdown_reports/{Tstamp}.md'
    with open(report_path, 'w') as f:
        f.write(''.join(md_content).replace('<br></br>', '\n'))
    
    if verbose == 1:
        print(f'Generated a report at ==> {report_path}')
        
    return report_path

In [3]:
report_path = generate_latest_fakes_report(PROJ_DIR, verbose=1)


Latest fakes:

['/AFHQ-WILD_training-runs/00003-AFHQ-WILD_custom-auto2-resumecustom/fakes003113.png',
 '/AFHQ_training-runs/00002-AFHQ_custom-auto2-resumecustom/fakes002211.png',
 '/metfaces_training-runs/00004-metfaces_custom-auto2-resumecustom/fakes000846.png',
 '/StanfordDogs_training-runs/00003-StanfordDogs_custom-auto2-resumecustom/fakes002826.png',
 '/cars196_training-runs/00012-cars196_custom-auto2-resumecustom/fakes004178.png',
 '/AFHQ-DOG_training-runs/00003-AFHQ-DOG_custom-auto2-resumecustom/fakes002457.png',
 '/102flowers_training-runs/00022-102flowers_custom-auto2-resumecustom/fakes004300.png',
 '/gen1_pokemon_training-runs/00007-gen1_pokemon_custom-auto2-resumecustom/fakes000086.png']


fakes003113.png Resized from (7680, 4096) [66.31MB] to ==> (1920, 1024) [4.88MB]
Link ==> https://i.ibb.co/6v293tx/5fd41961782c.png

fakes002211.png Resized from (7680, 4096) [62.36MB] to ==> (1920, 1024) [4.57MB]
Link ==> https://i.ibb.co/GP0MQv3/99879c53cfcc.png

fakes000846.png Resized 

'/work/chaselab/malyetama/ADA_Project/latest_fakes_markdown_reports/03_20_2021__15_15.md'

In [4]:
display(Markdown('/work/chaselab/malyetama/ADA_Project/latest_fakes_markdown_reports/03_20_2021__15_15.md'))

# Latest fakes
## Date and time: 03/20/2021, 15:14:53
### AFHQ-WILD_training-runs/00003-AFHQ-WILD_custom-auto2-resumecustom/fakes003113.png
![fakes003113.png](https://i.ibb.co/6v293tx/5fd41961782c.png "AFHQ-WILD_training-runs/00003-AFHQ-WILD_custom-auto2-resumecustom/fakes003113.png")

### AFHQ_training-runs/00002-AFHQ_custom-auto2-resumecustom/fakes002211.png
![fakes002211.png](https://i.ibb.co/GP0MQv3/99879c53cfcc.png "AFHQ_training-runs/00002-AFHQ_custom-auto2-resumecustom/fakes002211.png")

### metfaces_training-runs/00004-metfaces_custom-auto2-resumecustom/fakes000846.png
![fakes000846.png](https://i.ibb.co/Kr8sZd3/20404bdf027b.png "metfaces_training-runs/00004-metfaces_custom-auto2-resumecustom/fakes000846.png")

### StanfordDogs_training-runs/00003-StanfordDogs_custom-auto2-resumecustom/fakes002826.png
![fakes002826.png](https://i.ibb.co/BrR0y8s/ca928549c65e.png "StanfordDogs_training-runs/00003-StanfordDogs_custom-auto2-resumecustom/fakes002826.png")

### cars196_training-runs/00012-cars196_custom-auto2-resumecustom/fakes004178.png
![fakes004178.png](https://i.ibb.co/y4ZRzCh/1b8194c5b309.png "cars196_training-runs/00012-cars196_custom-auto2-resumecustom/fakes004178.png")

### AFHQ-DOG_training-runs/00003-AFHQ-DOG_custom-auto2-resumecustom/fakes002457.png
![fakes002457.png](https://i.ibb.co/LQM6W8L/d3f5136ade51.png "AFHQ-DOG_training-runs/00003-AFHQ-DOG_custom-auto2-resumecustom/fakes002457.png")

### 102flowers_training-runs/00022-102flowers_custom-auto2-resumecustom/fakes004300.png
![fakes004300.png](https://i.ibb.co/WW9kpTt/4ab5ddf20cb3.png "102flowers_training-runs/00022-102flowers_custom-auto2-resumecustom/fakes004300.png")

### gen1_pokemon_training-runs/00007-gen1_pokemon_custom-auto2-resumecustom/fakes000086.png
![fakes000086.png](https://i.ibb.co/KV3NqTF/fe501436248d.png "gen1_pokemon_training-runs/00007-gen1_pokemon_custom-auto2-resumecustom/fakes000086.png")

